<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #5: Adding new clips to a Zooniverse workflow</h1>
<h3 align="right">Written by @jannesgg</h3>

In [1]:
### Import modules

In [1]:
import getpass
import qgrid
from ipyfilechooser import FileChooser
import pandas as pd
from db_setup import add
from utils import db_utils

### Step 1: Reduce the size of the video and optionally blur sensitive portions
#### The first argument is the directory where movies are stored
*Note:  The original files are kept in case of any problems during processing with the suffix *_orig**

In [9]:
### Define important locations

In [10]:
# Find the folder with the appropriate movies
fc = FileChooser('/cephyr/NOBACKUP/groups/snic2021-6-9')
display(fc)

FileChooser(path='/cephyr/NOBACKUP/groups/snic2021-6-9', filename='', title='HTML(value='', layout=Layout(disp…

In [11]:
movies_location = fc.selected
movie_metadata_location = "https://drive.google.com/file/d/1IBBm4GqZGUZvnVJ3DbIHwJ_JQQW6sCEl/view?usp=sharing"
# Optional species metadata location if new species appear in movies
species_metadata_location = "https://drive.google.com/file/d/18_5h4fzX7zau-JltIRPoFrltJm_cbxyb/view?usp=sharing"
movie_extension = "mov" #if movies are not all of the same extension this will fail
blur_movies = 0 # (optional) if the videos contain sensitive information, use 1 here to blur the clips

In [5]:
!bash /usr/src/app/koster_data_management/ingestion_scripts/process_movies.sh \
        $movies_location $movie_extension $blur_movies

Processing video...None*
^C


### Step 2: Specify Zooniverse and database credentials

In [ ]:
# Specify username and password of a valid zooniverse account
user_zoo = getpass.getpass('Enter your Zooniverse user')
pass_zoo = getpass.getpass('Enter your Zooniverse password')

In [12]:
# Specify your database file location
db_path = "/usr/src/app/data_dir/koster_lab-nm-9.db"
#db_path = "/cephyr/NOBACKUP/groups/snic2021-6-9/db_files/tutorial_demo.db"

### Step 3: Add the new movies and new species (if applicable) to the Koster database

In [ ]:
# New movies
add.add_new_movies(movie_metadata_location, 
                   db_path,
                   movies_location)
# New species

add.add_species(species_metadata_location,
                db_path)

### Step 4: Identify movies of interest

#### We first need to find the ids of the movies we are interested in splitting into clips. We do this by looking at the movies table within the database. 

In [13]:
# Look over the available movies to choose the ones to extract clips from
conn = db_utils.create_connection(db_path)
movies_df = pd.read_sql_query("SELECT * FROM movies", conn)

### Step 5: Upload clips to Zooniverse

The script to upload clips to Zooniverse requires a specification of the total number of clips, the length of each clip, the list of video ids to extract these from and (optionally) the number of clips to be extracted from each video in the list. 

You may receive an error message related to file size if clips exceed the recommended limit for Zooniverse uploads. In this case, we recommend shortening the clip length to achieve a suitable filesize.

In [14]:
# Find movies of interest
qgrid.show_grid(movies_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [ ]:
# Specify clip-related parameters
clips_output_path = "/cephyr/NOBACKUP/groups/snic2021-6-9/clips_challenge_1" # where clips will be stored
n_clips = 980
clip_length = 10 # in seconds
video_list = [61, 62, 63, 64]
n_clips_each = [60, 60, 60, 800]

In [ ]:
# Run the script

In [ ]:
run = f"export LC_ALL='C.UTF-8' python upload_clips.py -u {user_zoo} -p {pass_zoo} -db {db_path} -fp {clips_output_path} \
     -n {n_clips} -lg {clip_length} -vlist {video_list} \
     -neach {n_clips_each}"
!{run}

In [ ]:
#### EVERYTHING BELOW THIS LINE IS STILL IN PROGRESS

## Adding new frames after clips have been classified

In [ ]:
# Look over the available species to choose the ones to extract frames from
conn = db_utils.create_connection(db_path)
species_df = pd.read_sql_query("SELECT * FROM species", conn)

In [ ]:
species_df.head()

In [ ]:
!export LC_ALL="C.UTF-8" python upload_frames.py --user $user_zoo --password $pass_zoo \
        --species "Fish (any species)" --db_path $db_path \
        -fp /cephyr/NOBACKUP/groups/snic2021-6-9/processed_frames/fish_any_species